# **1. Mô tả ứng dụng** 

## **Ứng dụng mà nhóm chọn**
Huấn luyện Convolutional Neural Network để phân lớp ảnh bằng Stochastic Gradient Descent.
*   **Input:** Tập dữ liệu ảnh và nhãn của ảnh, cấu trúc của CNN và các tham số liên quan.

*   **Output:** Mô hình phân lớp ảnh sau khi huấn luyện.

*   **Ý nghĩa của ứng dụng trong thực tế:** Giúp số hoá tài liệu, OCR...

## **Ứng dụng này có cần phải tăng tốc không**
Có, vì tập dữ liệu lớn và có nhiều hidden layer nên chạy rất chậm nếu chạy tuần tự.

## **Ứng dụng này có thể tăng tốc bằng cách song song hóa không**
Có, vì trong đó có phép convolution, phép nhân ma trận, phép max-pooling,...

# **2. Cài đặt tuần tự**

## **Thiết kế**

*   Đối với mỗi bức ảnh đầu vào, ta sẽ chuyển nó về mảng numpy.

*   Nếu là ảnh RGB thì ta sẽ chuyển nó về ảnh grayscale.

*   Trước khi đưa vào CNN ta sẽ chuẩn hoá mảng numpy trên về khoảng [-0.5,0.5] để tránh việc giá trị cách lớp sau trở nên quá lớn.

*   CNN sẽ có 1 convolutional layer, 1 max-pooling layer, 1 fully connected layer và activation là hàm softmax.

    *   Convolutional layer dùng để nhận dạng các đặc trưng cụ thể hoặc hình mẫu trong bức ảnh.

    *   Max-pooling layer dùng để tập hợp các đặc trưng nhỏ lẻ của ảnh thành một đặc trưng lớn hơn, ít nhạy cảm với vị trí của đặc trưng hơn.

    *   Fully connected layer dùng để quyết định nhãn của ảnh.

    ![](https://victorzhou.com/media/cnn-post/cnn-dims-3.svg)

    <center><small>Nguồn: https://victorzhou.com/blog/intro-to-cnns-part-1/</small></center>

*   **Lan truyền xuôi:**

    *   ***Convolutional layer:***

        Trong layer này, ta sẽ có $n$ node tương đương với $n$ filter với kích thước là $m$x$m$, trọng số của các filter này sẽ được khởi tạo ngẫu nhiên và mỗi trọng số đó sẽ được chia cho $m$x$m$ để chống việc kết quả ra số quá lớn.

        Ta sẽ lần lượt trượt các filter từ phải sang trái, từ trên xuống dưới, mỗi lần một pixel. Với mỗi pixel tương ứng trong cửa sổ trượt, ta sẽ nhân giá trị của nó với trọng số trong filter rồi cộng lại với nhau.

        ![](https://drive.google.com/uc?export=view&id=1riGOZlVlZdE-iAId6b23kaMLz7NncRT-)

        ![](https://drive.google.com/uc?export=view&id=1M7zZWBg4Hr_aJ7OJZ1t_Y4s48DuYUSkq)

    *   ***Max pool layer:***

        Với pool có kích thước là $p$x$p$, ta cũng lần lượt trượt các filter từ phải sang trái, từ trên xuống dưới, nhưng mỗi lần là $p$ pixel. Ta sẽ chọn ra giá trị lớn nhất trong cửa sổ trượt đó.

        ![](https://drive.google.com/uc?export=view&id=1soCPEqlOsBjZNTap4opnfFF5rehv0anc)

    *   ***Fully connected layer:***

        Ta sẽ có $k$ node tương đương với $k$ lớp cần phân, vì đây là fully connected layer nên tất cả các node của layer trước sẽ được nối với tất cả các node trong layer này. Tức là giá trị đầu ra của mỗi node sẽ được tính bằng công thức sau: $t = w * x + b$ với $t$ là giá trị đầu ra của mỗi node, $w$ là vector chứa trọng số của node đó, $x$ là vector các giá trị đầu ra của tất cả các node ở layer trước đó và $b$ là bias của node đó. Với $k$ node ta sẽ thu được một vector $k$ chiều và ta sẽ tính softmax của vector này, mỗi giá trị sẽ tương ứng với xác suất của mỗi lớp và ta sẽ chọn lớp nào có xác suất cao nhất.

*   **Lan truyền ngược:**

    Trong quá trình này ta sẽ lần lượt tính các gradient của từng layer bằng quy tắt mắc xích để cập nhật các trọng số và bias trong các layer đó.

    *   ***Fully connected layer:***

        Ta sẽ dùng độ đo cross-entropy để tính độ lỗi: $L=-\text{ln}(p_c)$ với $p_c$ là xác suất của lớp đúng $c$.

        Ta có thể dùng quy tắc mắc xích để tính các gradient sau:
        \begin{equation}
          \frac{\partial L}{\partial w} = \frac{\partial L}{\partial {\text{output}_\text{FCL}}} * \frac{\partial {\text{output}_\text{FCL}}}{\partial t} * \frac{\partial t}{\partial w}
        \end{equation}
        \begin{equation}
          \frac{\partial L}{\partial b} = \frac{\partial L}{\partial {\text{output}_\text{FCL}}} * \frac{\partial {\text{output}_\text{FCL}}}{\partial t} * \frac{\partial t}{\partial b}
        \end{equation}
        \begin{equation}
          \frac{\partial L}{\partial {\text{output}_\text{MPL}}} = \frac{\partial L}{\partial {\text{output}_\text{FCL}}} * \frac{\partial {\text{output}_\text{FCL}}}{\partial t} * \frac{\partial t}{\partial {\text{output}_\text{MPL}}}
        \end{equation}

        Stochastic Gradient Descent cũng giống như Gradient Descent nhưng thay vì cập nhật các trọng số và bias sau khi duyệt hết tập train, ta sẽ cập nhật với mỗi một mẫu được duyệt, cũng có thể hiểu là batch size = 1.

        Vì vậy ta sẽ cập nhật các trọng số sau mỗi một mẫu được duyệt: $w_i = w_i - \alpha * \frac{\partial L}{\partial w}$ với $w_i$ là trọng số thứ $i$ và $\alpha$ là hệ số học.

        Tương tự, ta cũng cập nhật các bias: $b_i = b_i - \alpha * \frac{\partial L}{\partial b}$ với $b_i$ là bias thứ $i$ và $\alpha$ là hệ số học.

        Cuối cùng ta truyền $\frac{\partial L}{\partial {\text{output}_\text{MPL}}}$ về cho max pool layer.

        Chi tiết các công thức có trong tài liệu tham khảo [2].

    *   ***Max pool layer:***

        Vì layer này không có trọng số nên ta không cần cập nhật gì cả. Ta chỉ cần khôi phục lại kích thước ban đầu trước khi đi qua max pool layer, rồi gán $\frac{\partial L}{\partial {\text{output}_\text{MPL}}}$ vào vị trí có giá trị lớn nhất trong cửa sổ trượt $p$x$p$ ban đầu, những vị trí còn lại sẽ được gán bằng 0. Đây chính là $\frac{\partial L}{\partial {\text{output}_\text{CL}}}$.

        ![](https://victorzhou.com/media/cnn-post/maxpool-backprop.svg)

        <center><small>Nguồn: https://victorzhou.com/blog/intro-to-cnns-part-2/</small></center>

        Ta sẽ truyền $\frac{\partial L}{\partial {\text{output}_\text{CL}}}$ về cho convolutional layer.

    *   ***Convolutional layer:***

        Ta cũng dùng quy tắc mắc xích để tính gradient sau:
        \begin{equation}
          \frac{\partial L}{\partial \text{filter}(x,y)} = \sum\limits_{i}\sum\limits_{j}\frac{\partial L}{\partial {\text{output}_\text{CL}(i,j)}} * \frac{\partial {\text{output}_\text{CL}(i,j)}}{\partial \text{filter}(x,y)}
        \end{equation}

        Ta cập nhật các trọng số của filter: $\text{filter}(x,y) = \text{filter}(x,y) - \alpha * \frac{\partial L}{\partial \text{filter}(x,y)}$ với $\text{filter}(x,y)$ là trọng số của filter ở dòng $x$ cột $y$ và $\alpha$ là hệ số học.

        Chi tiết các công thức có trong tài liệu tham khảo [2].

*   Lập lại cho tất cả các ảnh còn lại.
    
*   Nếu đã duyệt hết ảnh, ta sẽ trộn ngẫu nghiên các mẫu và duyệt lại từ đầu, việc này được gọi là epoch.


## **Đánh giá**

*   Thử nghiệm trên tập dữ liệu chữ số viết tay MNIST. Tập dữ liệu này chứa các ảnh chữ số viết tay từ 0 đến 9, nó bao gồm 60000 ảnh trong tập train và 10000 ảnh trong tập test, mỗi ảnh có kích thước là 28x28 và là ảnh grayscale.

*   Cấu hình của CNN như sau:

  *   Convolutional layer sẽ có 32 filter, mỗi filter sẽ có kích thước là 5x5.

  *   Max pool layer có kích thước của pool là 2x2.

  *   Fully connected layer sẽ có 10 node tương ứng với 10 chữ số từ 0 đến 9.

  *   $\alpha = 0.005$

  *   epoch = 100

*   Kết quả: 

  *   Thời gian huấn luyện khá lâu khoảng 2 tiếng 30 phút cho 100 epoch với mỗi epoch khoảng 90 giây. Tỷ lệ chính xác khi train khá cao khoảng 99% cũng như tỷ lệ chính xác khi test khoảng 97%.

  *   Thời gian huấn luyện của keras nhanh hơn khoảng 1 tiếng 55 phút cho 100 epoch với mỗi epoch khoảng 67 giây. Tỷ lệ chính xác khi train và khi test cao hơn khoảng 1%, 100% đối với khi train, 98% đối với khi test.

In [ ]:
!python source.py

Loading data...
Normalizing...
Initiating parameters...

--------------Our model--------------

Epoch 1/100
	95s - loss: 0.2124 - accuracy: 0.9360 - val_loss: 0.0912 - val_accuracy: 0.9726
Epoch 2/100
	90s - loss: 0.0976 - accuracy: 0.9703 - val_loss: 0.0731 - val_accuracy: 0.9773
Epoch 3/100
	90s - loss: 0.0794 - accuracy: 0.9763 - val_loss: 0.0731 - val_accuracy: 0.9751
Epoch 4/100
	90s - loss: 0.0697 - accuracy: 0.9788 - val_loss: 0.0718 - val_accuracy: 0.9778
Epoch 5/100
	90s - loss: 0.0623 - accuracy: 0.9803 - val_loss: 0.0837 - val_accuracy: 0.9723
Epoch 6/100
	90s - loss: 0.0587 - accuracy: 0.9815 - val_loss: 0.0766 - val_accuracy: 0.9778
Epoch 7/100
	90s - loss: 0.0533 - accuracy: 0.9837 - val_loss: 0.0773 - val_accuracy: 0.9759
Epoch 8/100
	90s - loss: 0.0518 - accuracy: 0.9835 - val_loss: 0.0684 - val_accuracy: 0.9812
Epoch 9/100
	90s - loss: 0.0481 - accuracy: 0.9846 - val_loss: 0.0819 - val_accuracy: 0.9777
Epoch 10/100
	90s - loss: 0.0468 - accuracy: 0.9854 - val_loss: 0.0

# **3. Cài đặt song song (trên GPU)**

*   **Phân tích:**
*   **Thiết kế:**
*   **Đánh giá:**

# **4. Cài đặt song song (trên GPU) + tối ưu hóa**

# **5. Nhìn lại quá trình làm đồ án**

# **6. Tài liệu tham khảo**

[1] https://victorzhou.com/blog/intro-to-cnns-part-1/

[2] https://victorzhou.com/blog/intro-to-cnns-part-2/

[3] https://www.kaggle.com/moltean/fruits